In [17]:
import warnings
warnings.filterwarnings('ignore')

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import classification_report , confusion_matrix , accuracy_score
from sklearn.model_selection import train_test_split

import cv2
#from google.colab.patches import cv2_imshow
from PIL import Image 
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Input, Dense,Conv2D , MaxPooling2D, Flatten,BatchNormalization,Dropout
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [18]:
current_dir = os.getcwd()  # Cambia esto a la ruta base de tu proyecto si es necesario

# Definir las rutas de las carpetas
normal_path = os.path.join(current_dir, 'data', 'normal')
stroke_path = os.path.join(current_dir, 'data', 'stroke')

# Comprobar si las carpetas existen
if os.path.exists(normal_path) and os.path.exists(stroke_path):
    normal_folder = os.listdir(normal_path)
    stroke_folder = os.listdir(stroke_path)

    data = []

    # Cargar imágenes de la carpeta Normal
    for img_file in normal_folder:
        try:
            image = Image.open(os.path.join(normal_path, img_file)).resize((224, 224)).convert('RGB')
            data.append(np.array(image))
        except Exception as e:
            print(f"No se pudo abrir la imagen {img_file} en Normal: {e}")

    # Cargar imágenes de la carpeta Stroke
    for img_file in stroke_folder:
        try:
            image = Image.open(os.path.join(stroke_path, img_file)).resize((224, 224)).convert('RGB')
            data.append(np.array(image))
        except Exception as e:
            print(f"No se pudo abrir la imagen {img_file} en Stroke: {e}")

    # Comprobar si se han cargado imágenes
    if data:
        print(f"Se han cargado {len(data)} imágenes.")
    else:
        print("No se han cargado imágenes.")
else:
    print("No se pueden cargar las imágenes porque las rutas no son válidas.")

No se pueden cargar las imágenes porque las rutas no son válidas.


In [10]:
normal_label = [0]*len(normal_folder)
stroke_label = [1]*len(stroke_folder)

Target_label = normal_label + stroke_label
len(Target_label)

x = np.array(data)
y = np.array(Target_label)

NameError: name 'normal_folder' is not defined

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.20,shuffle=True)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
x_train_s = x_train/255
x_test_s = x_test/255

In [ ]:
model = Sequential()

model.add(Conv2D(32,kernel_size=(3,3),strides=1,padding="valid",activation="relu", input_shape=(224,224,3)))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(64,kernel_size=(3,3),strides=1,padding="valid",activation="relu"))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(128,kernel_size=(3,3),strides=1,padding="valid",activation="relu"))
model.add(MaxPooling2D(pool_size=2))

model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(1, activation="sigmoid"))

model.compile(optimizer="adam",loss="binary_crossentropy",
             metrics=["accuracy"])

model.summary()

In [ ]:
history = model.fit(x_train_s,y_train,batch_size=32,
         epochs=10, validation_data=(x_test_s,y_test))

In [ ]:
loss, acc = model.evaluate(x_test_s,y_test)
print("Loss on Test Set:",loss)
print("Accuracy on Test Set:",acc)

loss, acc = model.evaluate(x_train_s,y_train)
print("Loss on Train Set:",loss)
print("Accuracy on Train Set:",acc)

In [ ]:
y_pred_test = model.predict(x_test_s)
y_pred_test_label = [1 if i>=0.5 else 0 for i in y_pred_test]
print("Actual Label:",y_test[:10])
print("Predicted Label:",y_pred_test_label[:10])

In [ ]:
print("-----Metrics Evaluation On Test Set -----")
print()
print("Confusion Matrix:\n",confusion_matrix(y_test,y_pred_test_label))
print()
print("Classification Report:\n",classification_report(y_test,y_pred_test_label))

In [ ]:
'''
current_dir = os.path.dirname(os.path.abspath(__file__))
model_path = os.path.join(current_dir, 'nn_stroke_img.keras')
model.save_model(model_path)
'''